EEG Spike-Wave visualization tool
Copyright (C) 2023  Leonardo Cañete-Sifuentes, Azul Salmerón Ruiz, Lizbeth Becerril Belio 

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

Repository: https://github.com/LeonardoC/SpikeWave-Visualization

In [1]:
import pandas as pd
from linelength_event_detector.lleventdetector import *
from linelength_event_detector.lltransform import *
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
data = pd.read_csv('C:/Users/L03109567/Desktop/LNHV- Análisis/E3,4,5,6/LB 1/LB 1.csv', header = None)
data = data.to_numpy()
data = data.transpose()

In [32]:
def detect_events(data, subject, sfx, mel, llw, prc):
  channel0 = subject * 2
  channel1 = channel0 + 1
  actual = lltransform(data[channel0], sfx, llw)
  events = lleventdetector([actual], sfx, prc, mel, llw)[0]
  channels = [channel0 for i in range(events.shape[0])]
  channels = np.array(channels, ndmin=2).transpose()
  events = np.hstack((events, channels))
  
  actual = lltransform(data[channel1], sfx, llw)
  new_events = lleventdetector([actual], sfx, prc, mel, llw)[0]
  channels = [channel1 for i in range(new_events.shape[0])]
  channels = np.array(channels, ndmin=2).transpose()
  new_events = np.hstack((new_events, channels))
  events = np.vstack((events, new_events))
  events = events[events[:, 0].argsort()]
  return events

def split_events(events):
  windows = []
  channels = []
  i = 0
  large_window = False
  while i < events.shape[0] - 1:
    if large_window:
      if events[i][0] > windows[-1][1]:
        large_window = False
      elif events[i][1] > windows[-1][1]:
        windows[-1][1] = events[i][1]
        i += 1
      else:
        i += 1

    elif events[i][1] < events[i+1][0]:
      windows.append((events[i][0], events[i][1]))
      channels.append((events[i][2],))
      i += 1
    else:
      large_window = True
      windows.append((events[i][0], events[i+1][1]))
      channels.append((events[i][2], events[i+1][2]))
      i += 2

  if large_window:
    if events[-1][1] > windows[-1][1]:
      windows[-1][1] = events[-1][1]

  else:
    windows.append((events[-1][0], events[-1][1]))
    channels.append((events[-1][2],))
  return windows, channels

def main(data, sfx, mel, llw, prc):
  events = [[],[]]
  for subject in range(4):
    subject_events = detect_events(data, subject, sfx, mel, llw, prc)
    subject_windows, subject_channels = split_events(subject_events)
    events[0] += subject_windows
    events[1] += subject_channels
  events[0] = np.array(events[0])
  return events

def print_results(data, res):
  for i in range(len(res[0])):
    chls = res[1][i]
    chls = [int(i) for i in chls]
    avgs = []
    for chl in chls:
      start =  int(res[0][i][0])
      end = int(res[0][i][1]) + 1
      avg = data[chl, start:end].mean()
      avgs.append(avg)
      time = (res[0][i][1] - res[0][i][0]) / 500
    print(i, res[0][i] / 500, time, "s, ", res[1][i], avgs)
    
def plot_window(data, sfx, windows, window):
    channels = windows[1][window]
    channels = [int(channel) for channel in channels]
    channels

    nchannels = len(channels)
    nsamples = data.shape[1]
    time = [i / sfx for i in range(nsamples)]

    start = int(windows[0][window][0])
    end = int(windows[0][window][1]) + 1
    
    fig = make_subplots(rows=nchannels, cols=1)
    for i in range(nchannels):
        fig.append_trace(go.Scatter(
            x=time[start:end],
            y=data[channels[i]][start:end],
            name= channels[i],
        ), row=i + 1, col=1)
    fig.update_layout(height=300*nchannels, width=600, showlegend= True)
    fig.show()

In [7]:
sfx = 500 # Sampling frequency
mel = 500 # Minimum event length (milliseconds)
prc = 99
llw = 1
events = main(data, sfx, mel, llw, prc)
print_results(data, events)

0 [11.576 12.63 ] 1.054 s,  (1.0, 0.0) [-0.0026702651515151518, 0.003762310606060606]
1 [12.822 13.836] 1.014 s,  (0.0,) [-0.004683858267716536]
2 [141.808 142.496] 0.688 s,  (0.0,) [-7.652173913043408e-05]
3 [144.338 145.24 ] 0.902 s,  (0.0, 1.0) [0.005608628318584071, -0.006038274336283187]
4 [425.728 427.624] 1.896 s,  (0.0, 1.0) [0.004773867228661749, -0.0013113804004214944]
5 [430.058 431.034] 0.976 s,  (0.0, 1.0) [0.00040306748466257907, 0.003734151329243355]
6 [455.862 456.558] 0.696 s,  (0.0, 1.0) [-0.001604871060171919, 0.014426074498567332]
7 [1689.264 1690.39 ] 1.126 s,  (0.0, 1.0) [0.002323049645390072, -0.005970744680851057]
8 [1691.856 1693.012] 1.156 s,  (1.0, 0.0) [-0.006163212435233162, 0.0029283246977547498]
9 [1728.986 1729.974] 0.988 s,  (0.0, 1.0) [-0.0028157575757575756, 0.007822828282828284]
10 [1771.558 1773.19 ] 1.632 s,  (1.0, 0.0) [0.003454100367197062, -9.755201958384455e-05]
11 [2235.754 2237.09 ] 1.336 s,  (0.0, 1.0) [0.0013497757847533627, 0.0049588938714

In [33]:
plot_window(data, sfx, events, 0)

In [26]:
def detect_hill(x):
    a, b = x
    return a <= 0 and b > 0
test_list = [4, 7, 3, 2, 9, 2, 1]
diffs = np.diff(data[0])
diffs

array([0.    , 0.    , 0.    , ..., 0.    , 0.    , 0.0006])

In [28]:
hills = list(map(detect_hill, zip(diffs[:-1], diffs[1:])))

In [29]:
sum(hills)

196962

In [14]:
data.shape

(8, 1815000)